# 목적
* 블로그 내용을 긁어와서 연관분석 진행
1. keyword관련 블로그 크롤링 
    정확도 기준으로 긁어오되 날짜도 같이 가져와서 나중 어떤 날짜에 집중되어 있는지 확인
    한 블로그당 단어빈도수 분석(필요할까)
2. 블로그 내 단어 토크나이즈하고 
3. 연관분석실시


In [3]:
# # 선행설치
# !pip install beautifulsoup4
# !pip install requests
# !pip install lxm
# # 연관분석
# !pip install apyori
# # 자연어처리
# !pip install konlpy (JDK 설치가 되어있어야함 )

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5984 sha256=5235bb621f4450c8a0ed214694dd194b7f3b0f6a4cf1cb568b9531188bd48e80
  Stored in directory: c:\users\try00\appdata\local\pip\cache\wheels\47\6f\0f\21a86f3679f7ed6bbe4dc6694f86818c5d85c2044bfab0f1e8
Successfully built apyori


In [16]:
import re
import json
import math
import datetime
import pandas as pd

import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup #크롤링

import konlpy
from konlpy.tag import Komoran #자연어처리
komoran = Komoran(userdic='./user_dictionary.txt')

from apyori import apriori #연관분석
from tqdm import tqdm_notebook #진행과정 시각화 

In [5]:
#페이지 카운트 수로 api호출된다
#1,000개일때 3분 
#100*100개 만개의 경우 "비정상적인 트래픽을 감지"해서 오류걸림 
naver_client_id = "uG5V8KhGUcRmRCtBzj7d"
naver_client_secret = "6MkFrc1bUG"
keyword = '양양'
display_count = 10
page_count = 200
sort_type = 'sim'

In [212]:
def get_blog_post(keyword, display_count, page_count, sort_type,save = True):
    '''
    keyword : 검색하고 싶은 키워드
    display_count : 한 페이지당 표출할 개수 min = 10, max = 100
    page_count : 총 진행할 페이지의 수 min = 1, max = 1000
    sort_type : 정렬옵션 "sim" (유사도순), "date" (날짜순)
    ''' 
    #날짜와 본문내용 저장
    postdates = []
    strings = []
    urls = []
    titles = []
    blogger_names = []
    
    encode_keyword = urllib.parse.quote(keyword)
    # get_blog_search_result_pagination_count로 처리할 수있는 페이지수를 1부터 페이지수까지 까지 각각 하나씩 긁어옴 
    for i in tqdm_notebook(range(1, page_count + 1),desc = "page work"):
        url = "https://openapi.naver.com/v1/search/blog?query=" + encode_keyword + "&display=" + str(
            display_count) + "&start=" + str(i) + "&sort=" + sort_type

        request = urllib.request.Request(url)

        request.add_header("X-Naver-Client-Id", naver_client_id)
        request.add_header("X-Naver-Client-Secret", naver_client_secret)

        response = urllib.request.urlopen(request)
        response_code = response.getcode()
         
        if response_code is 200:
            response_body = response.read()
            response_body_dict = json.loads(response_body.decode('utf-8'))
        #items의 개수만큼씩 진행
            for j in range(0, len(response_body_dict['items'])):
                blog_post_url = response_body_dict['items'][j]['link'].replace("amp;", "")
                urls.append(blog_post_url)
                #블로그 url안에 들어가기(아직 크롤링불가)
                get_blog_post_content_code = requests.get(blog_post_url)
                get_blog_post_content_text = get_blog_post_content_code.text
                get_blog_post_content_soup = BeautifulSoup(get_blog_post_content_text, 'lxml')
                #크롤링가능한 url에 접속
                real_blog_post_url = "http://blog.naver.com" + get_blog_post_content_soup.select('#mainFrame')[0].get('src')
                get_real_blog_post_content_code = requests.get(real_blog_post_url)
                get_real_blog_post_content_text = get_real_blog_post_content_code.text
                get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')
                #본문부분 추출 
                blog_post_content = get_real_blog_post_content_soup.select('div#postViewArea')
                if len(blog_post_content) == 0:
                    blog_post_content = get_real_blog_post_content_soup.select('div.se-main-container')

                #전체 텍스트 
                string = ""
                for sentence in blog_post_content[0].stripped_strings:
                    string += " "+sentence.replace('\xa0'," ")
                #공백제거 
                string = string.replace('\u200b'," ")
                string = string.replace('\n'," ")
                string = string.replace('😑','')
                strings.append([string]) 

                #포스트날짜
                postdate = datetime.datetime.strptime(response_body_dict['items'][j]['postdate'],"%Y%m%d").strftime("%y.%m.%d")
                postdates.append(postdate)
                
                #제목
                remove_html_tag = re.compile('<.*?>')
                title = re.sub(remove_html_tag, '', response_body_dict['items'][j]['title'])
                titles.append(title)
                
                #블로거이름
                blogger_name = response_body_dict['items'][j]['bloggername']
                blogger_names.append(blogger_name)
                
    # utf-8형식으로 저장하면 엑셀에서 열때 에러나지만, load는 가능
    if save == True:
        crawling_df = pd.DataFrame({"post_dates":postdates, "title":titles, "blogger_name":blogger_names, "full_text":strings, "url":urls})
        crawling_df.to_csv(keyword+"_"+str(display_count* page_count)+".csv",encoding='utf-8',index= False)
    return crawling_df

In [213]:
crawling_df = get_blog_post(keyword, display_count, page_count, sort_type)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


IndexError: list index out of range

In [174]:
crawling_df['full_text'][0]

'[" 오늘은 양양 서핑 으로 동해에서 가장 핫한 서핑샵 플리즈웨잇서프를 소개해 드리려고 해요! 양양은 강원도 내에서도 많이 유명한 지역이죠! 서퍼들이 많이 모이면서 동네 자체가 아주 힙하고 독특한 분위기에요. 제가 다녀온 곳은 양양 인구해변에 있는 플리즈웨잇 서프 입니다!   패키지 숙박이용 - 양양 솔게하 입실 PM 04:00 / 퇴실 AM 11:00 플리즈웨잇서프에 자주 방문을 하는 분들은 양양 서핑 과 강습 파티까지 패키지로 한꺼번에 즐기기 위해 오는 분들이 많다고 해요. 저 또한 패키지로 예약하고 연계 되어있는 솔게하 도미토리를 이용했어요. 양양 솔게하 도미토리 솔게하의 자랑거리는 전객실이 오션뷰였어요. 인구해변 바로 앞에 있는 건물에 위치해있어 객실에 들어가자마 멋진 바다를 함께 할 수 있었어요. 작은 고추가 맵다는 말이 있잖아요! 도미토리인데도 정말 좋았어요. 드라이기,고데기,화장대 다 있습니다! 굳이 챙겨안와도 되요. 흰 타일바닥으로 되어있어 청결한 느낌을 받을 수 있었어요. 내부 안에서 찍은 뷰인데 너무 아름답지 않나요? 사장님께서도 작은방도 신경을 쓴것을 느낄 수 있었어요. 입실을 하게 되면 1인당 수건과 칫솔 귀마개 주세요. 파티 참석을 할 경우 1차 바베큐파티 팔찌 2차 펍파티용 나눠서 주시는데 잃어버리지 않도록 주의하셔야해요! 혹시 몰라 저는 받자마자 손에 끼어놨어요. 도미토리는 콘센트가 부족할경우 불편하잖아요. 이곳은 침대마다 배치가 되어있어 걱정없이 사용을 할 수 있었어요. 객실에도 있고 화장실 등등 많아 모르는분과 함께 쓰게 되도 얼굴 붉힐일이 없을 것 같아 개인적으로 좋았어요. 건물이 7층까지 되어있어 엘레베이터도 있었어요. 캐리어를 들고 갔었는데 짐을 들고 올라가기 편했어요 ※주의사항&꿀팁※ 솔게하 도미토리 입실 시간은 오후 4시에요~! 입실 당일 양양 서핑 을 하러 바로 바다에 가시는 분들이 많으신데 공용샤워장에서 씻고 오셔야 체크인 가능! 짐은 로비에 맡길 수 있다고 해요 \'플리즈웨잇서프 서핑강습\' 서핑강습은 로비에 시간

### 연관분석

In [2]:
crawling_df = pd.read_csv('양양_1000.csv',encoding='utf-8')

In [211]:
komoran = Komoran(userdic='user_dictionary.txt')

#이모티콘 제거 [출처](https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python)

In [212]:
emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

#분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')

In [220]:
sw = ['오늘','시간','최근','지난','관련','이번','이날','현재','10','올해','지난해','있다','이상','기준','때문',
     "메뉴","주문","양양","강원도",'정도','도착','생각','양양군','사진','위치','바다','해변']
all_nouns =[]

for i in tqdm_notebook(range(0,len(crawling_df))):
    try:
        nouns = komoran.nouns(crawling_df['full_text'][i])
    except :
        nouns = emoji_pattern.sub(r'',crawling_df['full_text'][399])
        nouns = han.sub(r'',TEXT_refine)
    clean_nouns = list(filter(lambda word : (len(word)>1)&(word not in sw),nouns))
    all_nouns.append(clean_nouns)

C:\Users\try00\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [221]:
noun_df = pd.DataFrame({'nouns':all_nouns})

In [222]:
rules = apriori(all_nouns, min_support=0.2, min_condience =0.3)

In [223]:
results  = list(rules)

In [224]:
results_df = pd.DataFrame(columns =["lhs",'rhs','support','confidence','lift'])
index =0
for row in results:   
    support = row[1]
    ordered_stat = row[2]
    for ordered_item in ordered_stat:
        lhs = " ".join(x for x in ordered_item[0])
        rhs = " ".join(x for x in ordered_item[1])
        confidence = ordered_item[2]
        lift = ordered_item[3]
        results_df.loc[index] = [lhs,rhs,support,confidence,lift]
        index = index +1

In [228]:
results_df = results_df.sort_values(by="confidence",ascending=False)

In [229]:
results_df

,lhs,rhs,support,confidence,lift
26,,여행,0.423,0.423,1.0
20,,사람,0.410,0.410,1.0
35,,코로나,0.379,0.379,1.0
25,,아침,0.311,0.311,1.0
15,,마음,0.308,0.308,1.0
13,,느낌,0.292,0.292,1.0
16,,맛집,0.290,0.290,1.0
36,,풍경,0.288,0.288,1.0
10,,기분,0.279,0.279,1.0
19,,보니,0.275,0.275,1.0
